In [1]:
import re
from vnpy.trader.constant import Exchange
from vnpy.app.cta_strategy.base import EXCHANGE_SYMBOL_DICT
from datetime import datetime, timedelta

In [36]:
fromDate = datetime.strptime('2012-6-10', '%Y-%m-%d')
toDate = datetime.strptime('2017-12-01', '%Y-%m-%d')

count = 0
while fromDate.date() != toDate.date():
    count += 1
    fromDate = fromDate + timedelta(1)

print(count)

2000


In [16]:
# 当前时间戳
import time
a = int(time.time())
print(a)

1563864223


In [59]:
# 时间戳转时间
timeStamp = 1512136800
date = datetime.fromtimestamp(timeStamp)
print(date)

2017-12-01 22:00:00


In [23]:
# 时间转时间戳
a = '2019-07-20 08:00:00'
b = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
print(b.timestamp())

1563580800.0


In [1]:
import requests
URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges'
response = requests.get(URL_EXCHANGES).json()
data = response['Data']
print(data.keys())

In [1]:
from pymongo import MongoClient, ASCENDING, DESCENDING
from vnpy.app.cta_strategy.base import TICK_DB_NAME, MINUTE_DB_NAME, DAILY_DB_NAME, MinuteDataBaseName, HourDataBaseName
from datetime import datetime
from vnpy.trader.object import BarData, TickData

In [54]:
# ====== Tick ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
symbol_list = ['BTCUSD', 'ETHUSD']
for symbol in symbol_list:
    cl = dbTick[symbol]
    cl.create_index('datetime')         # 添加索引
    start = datetime.strptime('2020-6-22 06:00:00', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2020-6-22 08:15:50', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = cl.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
        tick.__dict__ = dic
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}')
        count += 1
        if count >= 1:
            break

vt_symbol:BTCUSD.BYBIT	2020-06-23 12:11:33.868000	BTCUSD	9641.5
vt_symbol:ETHUSD.BYBIT	2020-06-23 12:11:33.844000	ETHUSD	242.4


In [41]:
# ====== Minute ======
client = MongoClient('localhost', 27017)
db = client[MINUTE_DB_NAME]
collection = db['BTCUSD.BYBIT']
#collection = db['ZN2006']
start = datetime.strptime('2020-3-24 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2020-3-25 00:00:00', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = collection.find().sort('datetime', DESCENDING)
count = 0
for dic in cursor:
    print(dic['datetime'], dic['open_price'])
    count += 1
    if count >= 6:
        break

2020-06-23 08:00:00 9693.0
2020-06-23 07:59:00 9693.5
2020-06-23 07:58:00 9698.0
2020-06-23 07:57:00 9698.0
2020-06-23 07:56:00 9696.0
2020-06-23 07:55:00 9696.0


In [5]:
# ====== Daily ======
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
symbol_list = ['BTCUSD.BYBIT', 'ETHUSD.BYBIT', 'EOSUSD.BYBIT']
#symbol_list = ['BTC.USDT.OKEX', 'ETH.USDT.OKEX', 'EOS.USDT.OKEX']
#symbol_list = ['Poloniex/BTC.USDT', 'Poloniex/ETH.USDT']
#symbol_list = ['Poloniex/BTC.USDT', 'Poloniex/ETH.USDT', 'Poloniex/EOS.USDT']
for symbol in symbol_list:
    collection = db[symbol]
    start = datetime.strptime('2019-11-27 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-27 00:06:00', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        symbol = dic['symbol']
        the_datetime = dic['datetime']
        open_price = dic['open_price']
        high_price = dic['high_price']
        low_price = dic['low_price']
        close_price = dic['close_price']
        print(f'{symbol}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
        count += 1
        if count >= 6:
            break
    print('\n')

BTCUSD.BYBIT	2020-06-15 08:00:00	9331.0	9501.0	8901.0	9422.5
BTCUSD.BYBIT	2020-06-14 08:00:00	9474.5	9480.0	9238.5	9331.0
BTCUSD.BYBIT	2020-06-13 08:00:00	9459.0	9500.0	9340.5	9474.5
BTCUSD.BYBIT	2020-06-12 08:00:00	9263.0	9545.0	9223.0	9459.0
BTCUSD.BYBIT	2020-06-11 08:00:00	9889.0	9987.0	9040.0	9263.0
BTCUSD.BYBIT	2020-06-10 08:00:00	9783.0	10038.0	9700.0	9889.0


ETHUSD.BYBIT	2020-06-15 08:00:00	231.7	233.8	217.85	230.95
ETHUSD.BYBIT	2020-06-14 08:00:00	238.1	238.5	229.8	231.7
ETHUSD.BYBIT	2020-06-13 08:00:00	237.65	238.8	234.6	238.1
ETHUSD.BYBIT	2020-06-12 08:00:00	230.1	239.4	228.1	237.65
ETHUSD.BYBIT	2020-06-11 08:00:00	247.9	250.6	225.15	230.1
ETHUSD.BYBIT	2020-06-10 08:00:00	244.15	251.2	241.55	247.9






In [3]:
symbol_list = ['RB2005', 'HC2005', 'SM2005', 'J2005', 'ZC2005', 'TA2005',
              'RB99', 'HC99', 'SM99', 'J99', 'ZC99', 'TA99']
symbol_list = ['ETHUSD.BYBIT']
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
for symbol in symbol_list:
    collection = db[symbol]
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        count += 1
        if count == 1:
            symbol = dic['symbol']
            exchange = dic['exchange']
            the_datetime = dic['datetime']
            open_price = dic['open_price']
            high_price = dic['high_price']
            low_price = dic['low_price']
            close_price = dic['close_price']
            print(f'{symbol}\t{exchange}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
            break
        
    print('\n')

ETHUSD.BYBIT	None	2020-06-15 08:00:00	231.7	233.8	217.85	230.95




In [59]:
# ====== 检查Tick质量 ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
cl = dbTick['BTCUSD']
cl.create_index('datetime')         # 添加索引
start = datetime.strptime('2019-12-20 07:59:50', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-12-30 08:00:10', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = cl.find(flt).sort('datetime')
count = 0
for dic in cursor:
    tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
    tick.__dict__ = dic
    if tick.last_price == 0:
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}\t{tick.ask_price_1}\t{tick.bid_price_1}')
    count += 1
print(count)

IndentationError: expected an indented block (<ipython-input-59-025e50492adc>, line 18)

In [4]:
a = 'abc'
try:
    print(a)
except:
    print('yes')
    pass
print(a)

abc
abc
